In [201]:
import googleapiclient.discovery

In [202]:
PROJECT = 'mlops-dev-env'
BUCKET =  'mlops-dev-workspace'
GCS_DATA_LOCATION = 'gs://workshop-datasets/covertype/data_validation'
REGION = 'us-central1'
LOCAL_WORKSPACE = '/home/jupyter/workspace'
LOCAL_DATA_DIR = os.path.join(LOCAL_WORKSPACE, 'data')
BQ_DATASET_NAME = 'data_validation'
BQ_TABLE_NAME = 'covertype_classifier_logs_tf'
MODEL_NAME = 'covertype_classifier_tf'
VERSION_NAME = 'v2'
TRAINING_DIR = os.path.join(LOCAL_WORKSPACE, 'training')
MODEL_DIR = os.path.join(TRAINING_DIR, 'exported_model')

In [203]:
instances = [
      { 
        'Soil_Type': ['7202'],
        'Wilderness_Area': ['Commanche'],
        'Aspect': [61],
        'Elevation': [3091],
        'Hillshade_3pm': [129],
        'Hillshade_9am': [227],
        'Hillshade_Noon': [223],
        'Horizontal_Distance_To_Fire_Points': [2868],
        'Horizontal_Distance_To_Hydrology': [134],
        'Horizontal_Distance_To_Roadways': [0], 
        'Slope': [8], 
        'Vertical_Distance_To_Hydrology': [10],
    }
]


request_body={
      'signature_name': 'serving_default',
      'instances': instances}

In [204]:
service = googleapiclient.discovery.build('ml', 'v1')
name = 'projects/{}/models/{}/versions/{}'.format(PROJECT, MODEL_NAME, VERSION_NAME)
print("Service name: {}".format(name))

Service name: projects/mlops-dev-env/models/covertype_classifier_tf/versions/v2


In [205]:
request_body

{'signature_name': 'serving_default',
 'instances': [{'Soil_Type': ['7202'],
   'Wilderness_Area': ['Commanche'],
   'Aspect': [61],
   'Elevation': [3091],
   'Hillshade_3pm': [129],
   'Hillshade_9am': [227],
   'Hillshade_Noon': [223],
   'Horizontal_Distance_To_Fire_Points': [2868],
   'Horizontal_Distance_To_Hydrology': [134],
   'Horizontal_Distance_To_Roadways': [0],
   'Slope': [8],
   'Vertical_Distance_To_Hydrology': [10]}]}

In [206]:
request = service.projects().predict(
      name=name,
      body=request_body
)

In [207]:
request

In [208]:
dir(request)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_in_error_state',
 '_process_response',
 '_rand',
 '_sleep',
 'add_response_callback',
 'body',
 'body_size',
 'execute',
 'from_json',
 'headers',
 'http',
 'method',
 'methodId',
 'next_chunk',
 'postproc',
 'response_callbacks',
 'resumable',
 'resumable_progress',
 'resumable_uri',
 'to_json',
 'uri']

In [209]:
request.method

'POST'

In [210]:
request.body

'{"signature_name": "serving_default", "instances": [{"Soil_Type": ["7202"], "Wilderness_Area": ["Commanche"], "Aspect": [61], "Elevation": [3091], "Hillshade_3pm": [129], "Hillshade_9am": [227], "Hillshade_Noon": [223], "Horizontal_Distance_To_Fire_Points": [2868], "Horizontal_Distance_To_Hydrology": [134], "Horizontal_Distance_To_Roadways": [0], "Slope": [8], "Vertical_Distance_To_Hydrology": [10]}]}'

In [211]:
request.headers

{'accept': 'application/json',
 'accept-encoding': 'gzip, deflate',
 'user-agent': '(gzip)',
 'x-goog-api-client': 'gdcl/1.8.0 gl-python/3.7.6',
 'content-type': 'application/json'}

In [16]:
request.uri

'https://ml.googleapis.com/v1/projects/mlops-dev-env/models/covertype_classifier_tf/versions/v2:predict?alt=json'

In [17]:
request.to_json()

'{"uri": "https://ml.googleapis.com/v1/projects/mlops-dev-env/models/covertype_classifier_tf/versions/v2:predict?alt=json", "method": "POST", "body": "{\\"signature_name\\": \\"serving_default\\", \\"instances\\": [{\\"Soil_Type\\": [\\"7202\\"], \\"Wilderness_Area\\": [\\"Commanche\\"], \\"Aspect\\": [61], \\"Elevation\\": [3091], \\"Hillshade_3pm\\": [129], \\"Hillshade_9am\\": [227], \\"Hillshade_Noon\\": [223], \\"Horizontal_Distance_To_Fire_Points\\": [2868], \\"Horizontal_Distance_To_Hydrology\\": [134], \\"Horizontal_Distance_To_Roadways\\": [0], \\"Slope\\": [8], \\"Vertical_Distance_To_Hydrology\\": [10]}]}", "headers": {"accept": "application/json", "accept-encoding": "gzip, deflate", "user-agent": "(gzip)", "x-goog-api-client": "gdcl/1.8.0 gl-python/3.7.6", "content-type": "application/json"}, "methodId": "ml.projects.predict", "resumable": null, "response_callbacks": [], "_in_error_state": false, "body_size": 404, "resumable_uri": null, "resumable_progress": 0}'

In [192]:
uri = 'https://ml.googleapis.com/v1/projects/mlops-dev-env/models/covertype_classifier_tf/versions/v2:predict'

In [193]:
uri

'https://ml.googleapis.com/v1/projects/mlops-dev-env/models/covertype_classifier_tf/versions/v2:predict'

## Invoke the CAIPP service

In [226]:
instances = [
      { 
        'Soil_Type': ['7202'],
        'Wilderness_Area': ['Commanche'],
        'Aspect': [61],
        'Elevation': [3091],
        'Hillshade_3pm': [129],
        'Hillshade_9am': [227],
        'Hillshade_Noon': [223],
        'Horizontal_Distance_To_Fire_Points': [2868],
        'Horizontal_Distance_To_Hydrology': [134],
        'Horizontal_Distance_To_Roadways': [0], 
        'Slope': [8], 
        'Vertical_Distance_To_Hydrology': [10],
    }
]
instances

[{'Soil_Type': ['7202'],
  'Wilderness_Area': ['Commanche'],
  'Aspect': [61],
  'Elevation': [3091],
  'Hillshade_3pm': [129],
  'Hillshade_9am': [227],
  'Hillshade_Noon': [223],
  'Horizontal_Distance_To_Fire_Points': [2868],
  'Horizontal_Distance_To_Hydrology': [134],
  'Horizontal_Distance_To_Roadways': [0],
  'Slope': [8],
  'Vertical_Distance_To_Hydrology': [10]}]

In [235]:
import json
body = {'instances': instances}
body = json.dumps(body)
body

'{"instances": [{"Soil_Type": ["7202"], "Wilderness_Area": ["Commanche"], "Aspect": [61], "Elevation": [3091], "Hillshade_3pm": [129], "Hillshade_9am": [227], "Hillshade_Noon": [223], "Horizontal_Distance_To_Fire_Points": [2868], "Horizontal_Distance_To_Hydrology": [134], "Horizontal_Distance_To_Roadways": [0], "Slope": [8], "Vertical_Distance_To_Hydrology": [10]}]}'

In [239]:
#uri = 'https://ml.googleapis.com/v1/projects/mlops-dev-env/models/covertype_classifier_tf/versions/v2:predict?alt=json'
uri = 'https://ml.googleapis.com/v1/projects/mlops-dev-env/models/covertype_classifier_tf/versions/v2:predict'
uri

'https://ml.googleapis.com/v1/projects/mlops-dev-env/models/covertype_classifier_tf/versions/v2:predict'

In [240]:
headers = 'Content-Type:application/json'
service_account_email = 'caipp-caller@mlops-dev-env.iam.gserviceaccount.com'
scope = 'https://www.googleapis.com/auth/cloud-platform'

In [241]:
!gcloud tasks create-http-task --queue=caip-load-generator \
--url={uri} \
--header={headers} \
--body-content='{body}' \
--oauth-service-account-email={service_account_email} \
--oauth-token-scope={scope}

Created task [projects/mlops-dev-env/locations/us-central1/queues/caip-load-generator/tasks/12392564199719055351].
